In [ ]:

import pandas as pd
import colorcet as cc
import polyline
import math
import numpy as np
colour_palette = cc.CET_C6s

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, TextInput, Tooltip
from bokeh.io import output_notebook, reset_output

output_notebook()

### making data a bit easier to see
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### should be outside the repo'
data_location = r'/Users/jj/code/strava_data/'

In [ ]:
### will return a list of colours of n length from the colour_palette
def get_colours(n):
    distance = int(len(colour_palette) / n)
    return [x for i, x in enumerate(colour_palette) if i % distance == 0]

def create_colour_map(s):
    unique = np.sort(s.unique())
    colours = get_colours(len(unique))
    return dict(zip(unique, colours))

In [ ]:
raw = pd.read_csv(data_location+'all_activities_cleaned.csv', index_col=0, parse_dates=True)
raw = raw.apply(lambda x: pd.to_datetime(x) if 'date' in x.name else x)
raw

In [ ]:
def geographic_to_web_mercator(x_lon, y_lat):     
    if abs(x_lon) <= 180 and abs(y_lat) < 90:          
        num = x_lon * 0.017453292519943295         
        x = 6378137.0 * num         
        a = y_lat * 0.017453292519943295          
        x_mercator = x         
        y_mercator = 3189068.5 * math.log((1.0 + math.sin(a)) / (1.0 - math.sin(a)))         
        return x_mercator, y_mercator      
    else:         
        print('Invalid coordinate values for conversion')

map_data = raw.copy()
map_data = map_data.dropna(subset=['map_summary_polyline'])
map_data['polyline'] = map_data['map_summary_polyline'].apply(polyline.decode)
map_data['mercator'] = map_data['polyline'].apply(lambda x: [geographic_to_web_mercator(y[1], y[0]) for y in x])

def custom_activity_type(a):
    if a['bike_name'] in ['Transition Smuggler', 'Canyon Spectral', 'Fat Bike']: return 'Mountain Bike Ride'
    elif a['bike_name'] in ['Avanti Giro 2', 'Cervelo Aspero']: return 'Road/Gravel Ride'
    elif a['type'] == 'Bike': return 'Ride Unknown'
    elif a['type'] == 'Run': return 'Run'
    elif a['type'] in ['Walk', 'Hike']: return 'Walk'
    else: return a['type']

map_data['activity_group'] = map_data.apply(custom_activity_type, axis=1)
map_data


In [ ]:
p = figure(width=800, height=800, x_axis_type="mercator", y_axis_type="mercator")
p.add_tile("CartoDB Positron")
for line in map_data[['mercator', 'sport_type', 'sport_type_colour']].values:
    x = [a[0] for a in line[0]]
    y = [a[1] for a in line[0]]

    p.line(x, y, color=line[2])

show(p)

In [ ]:
def categorical_map(cat_col):
    df = map_data.dropna(subset=[cat_col])[['mercator', cat_col]]
    df['colour'] = df[cat_col].map(create_colour_map(df[cat_col]))

    p = figure(width=800, height=800, x_axis_type="mercator", y_axis_type="mercator")
    p.add_tile("CartoDB.Positron")
    for activity in df.values:
        x = [a[0] for a in activity[0]]
        y = [a[1] for a in activity[0]]
        p.line(x, y, line_width=2, color=activity[2], line_alpha=.5, legend_label=activity[1])
    
    p.legend.click_policy="hide"

    show(p)

categorical_map('activity_group')

In [ ]:
import xyzservices.providers as xyz
xyz